In [ ]:
!git clone https://github.com/cknd/pyESN.git

Cloning into 'pyESN'...
remote: Enumerating objects: 106, done.
remote: Total 106 (delta 0), reused 0 (delta 0), pack-reused 106 (from 1)
Receiving objects: 100% (106/106), 1.17 MiB | 4.55 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
!pip install pyESN

In [ ]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=e94db87cc3071b81a1801377c14029f7db39cbcdec9904af421861851b350028
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [ ]:
import sys
sys.path.append('/content/pyESN')

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from google.colab import drive
from datetime import datetime
from pyswarm import pso
from pyESN import ESN
# from pyESN.pyESN import ESN
from math import sqrt
import pandas as pd
import numpy as np

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wc_real/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'YZV_data.csv'
HV_data = pd.read_csv(path + filename)

In [ ]:
set_index(HV_data)

In [ ]:
tm_data = HV_data
HV_data = HV_data.drop(columns=['YZV'])
HV_data['YZV'] = tm_data['YZV']
HV_data

,djia,interest_rate,inflation_rate,gdp_growth,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,...,"('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)",gold,vix,unemployment_rate,YZV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,-1.155684e+09,-312.491742,-280507.141339,-1.021796e+09,-9.532198,-115483.935741,-25.204908,-667.139276,-20.309269,-870.717996,...,-0.001971,-0.001822,-0.001782,-0.001782,-0.001782,-0.001782,-7.303430e+05,-5074.430765,-164.888872,-0.098637
2000-02-03,-1.038337e+09,-294.152654,-260330.361056,-9.471128e+08,-8.790286,-107723.491598,-23.159152,-618.146493,-18.955134,-799.604126,...,-0.001745,-0.001612,-0.001519,-0.001519,-0.001519,-0.001519,-6.766794e+05,-4871.204473,-152.205474,-0.089867
2000-02-04,-7.639965e+08,-230.976259,-200834.042712,-7.296305e+08,-6.727301,-83922.602134,-17.659121,-475.895763,-14.738414,-610.423502,...,-0.001302,-0.001217,-0.001076,-0.001076,-0.001076,-0.001076,-5.209539e+05,-3989.985998,-116.610418,-0.068001
2000-02-07,-4.803768e+08,-161.769260,-136734.041748,-4.955350e+08,-4.500745,-57854.216936,-11.749618,-322.833213,-10.139957,-410.827912,...,-0.000863,-0.000784,-0.000678,-0.000678,-0.000678,-0.000678,-3.535870e+05,-2979.521889,-78.602644,-0.045380
2000-02-08,-2.344655e+08,-97.924380,-78129.198414,-2.814042e+08,-2.468626,-33721.478802,-6.356898,-182.858321,-5.895064,-230.130383,...,-0.000364,-0.000368,-0.000311,-0.000311,-0.000311,-0.000311,-2.003376e+05,-1886.674367,-43.682933,-0.024180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,7.146629e+09,105.955953,367959.138182,3.061900e+09,4.906428,75042.327304,7.151391,185.257251,6.351472,1524.776491,...,-0.001023,-0.000502,-0.000690,-0.000690,-0.000690,-0.000690,2.575879e+07,-118.886061,65.925473,0.026248
2024-08-27,3.522269e+09,45.289956,157132.066741,1.308783e+09,2.221002,29914.845185,3.476358,76.788614,2.415067,774.080957,...,-0.001244,-0.000717,-0.000899,-0.000899,-0.000899,-0.000899,1.201494e+07,-381.296847,28.527492,0.006282
2024-08-28,-9.442316e+08,-27.959101,-97335.136463,-8.079582e+08,-1.051394,-23800.733155,-1.072689,-53.866326,-2.260864,-172.154813,...,-0.001311,-0.000852,-0.001012,-0.001012,-0.001012,-0.001012,-4.676237e+06,-746.562442,-16.834312,-0.015856


# Split train, validation and test data

In [ ]:
# def rolling_window_transform(data, in_sample_size, out_of_sample_size):
#   total_size = in_sample_size + out_of_sample_size
#   in_sample = []
#   out_of_sample = []

#   for start in range(len(data) - total_size + 1):
#     in_sample.append(data.iloc[start : start + in_sample_size])
#     out_of_sample.append(data.iloc[start + in_sample_size : start + total_size])

#   return np.array(in_sample), np.array(out_of_sample)

# in_sample_size = 100
# out_of_sample_size = 5

# train_in_sample, train_out_of_sample = rolling_window_transform(train_data, in_sample_size, out_of_sample_size)
# val_in_sample, val_out_of_sample = rolling_window_transform(val_data, in_sample_size, out_of_sample_size)
# test_in_sample, test_out_of_sample = rolling_window_transform(test_data, in_sample_size, out_of_sample_size)

In [ ]:
# train_end = datetime(2020, 6, 30)
# test_start = datetime(2020, 10, 1)

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  X_train = train_data.drop(columns=[target])
  y_train = train_data[target]

  X_val = val_data.drop(columns=[target])
  y_val = val_data[target]

  X_test = test_data.drop(columns=[target])
  y_test = test_data[target]

  return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## YZV

In [ ]:
HV_X_train, HV_y_train, HV_X_val, HV_y_val, HV_X_test, HV_y_test = split_data(HV_data, 'YZV')

In [ ]:
def esn_model(params, X_train, y_train, X_val):
  n_reservoir, spectral_radius, sparsity = int(params[0]), params[1], params[2]
  n_inputs = X_train.shape[1]
  n_outputs = 1

  esn = ESN(n_inputs=n_inputs, n_reservoir=n_reservoir, n_outputs=n_outputs, spectral_radius=spectral_radius, sparsity=sparsity)
  esn.fit(X_train, y_train)

  # Predict on validation data (for validation performance evaluation)
  pred_val = esn.predict(X_val)

  return pred_val

In [ ]:
def pso_objective(params):
  val_pred = esn_model(params, HV_X_train, HV_y_train, HV_X_val)
  loss = mean_squared_error(HV_y_val, val_pred)

  return loss

In [ ]:
# Initialize bounds for the parameters
lb = [10, 0.1, 0.01]  # Lower bounds for n_reservoir, spectral_radius, sparsity
ub = [500, 1.0, 0.5]  # Upper bounds for n_reservoir, spectral_radius, sparsity
swarmsize = 50
maxiter = 100

In [ ]:
optimal_params, _ = pso(pso_objective, lb, ub, swarmsize=swarmsize, maxiter=maxiter)

Stopping search: maximum iterations reached --> 100


In [ ]:
optimal_params

array([10.        ,  0.60569942,  0.49419789])

In [ ]:
# optimal_params = [4.83654438e+02, 2.14721646e-01, 4.11926862e-01]

In [ ]:
# Train and predict with optimal parameters
HV_test_predictions = esn_model(optimal_params, HV_X_train, HV_y_train, HV_X_test)

In [ ]:
HV_test_predictions.shape

(567, 1)

In [ ]:
HV_y_test.shape

(567,)

In [ ]:
mse = mean_squared_error(HV_y_test, HV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.030475149153430234
RMSE: 0.17457132970058467
MAE: 0.06479381503444012
MAPE: 58666.19030238714%


In [ ]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/YZV/'

# df_pred = pd.DataFrame(HV_test_predictions, columns=['esn_pred'])
# filename = 'ESN_pred.csv'
# df_pred.to_csv(path_1 + filename)